In [26]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import koreanize_matplotlib

import datetime
import random
import os
import sys


import sklearn
import catboost
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [14]:
def seed_everything(seed: int = 2024):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
seed_everything(2024)

In [15]:
train = pd.read_csv("./preprocessing_train.csv")
submission = pd.read_csv("./sample_submission.csv")

In [16]:
train.isnull().sum()

일시      0
최고기온    0
최저기온    0
일교차     0
강수량     0
평균습도    0
평균풍속    0
일조합     0
일사합     0
일조율     0
평균기온    0
dtype: int64

In [17]:
train["일시"] = pd.to_datetime(train["일시"])
submission["일시"] = pd.to_datetime(submission["일시"])

In [18]:
#년/월/일 추가
train['년']=train['일시'].dt.year
train['월']=train['일시'].dt.month
train['일']=train['일시'].dt.day

submission['년']=submission['일시'].dt.year
submission['월']=submission['일시'].dt.month
submission['일']=submission['일시'].dt.day

In [19]:
# 날짜 데이터
date_data = pd.date_range(start='1960-01-01', end='2022-12-31', freq='D')

# 일 단위 추출
day_of_year = date_data.dayofyear

# 주기 함수로 변환
train['Day sin'] = np.sin(2 * np.pi * day_of_year / 365)
train['Day cos'] = np.cos(2 * np.pi * day_of_year / 365)

# 날짜 데이터
date_data2 = pd.date_range(start='2023-01-01', end='2023-12-24', freq='D')

# 일 단위 추출
day_of_year2 = date_data2.dayofyear

# 주기 함수로 변환
submission['Day sin'] = np.sin(2 * np.pi * day_of_year2 / 358)
submission['Day cos'] = np.cos(2 * np.pi * day_of_year2 / 358)

In [20]:
# 주차 변수 추가
train["주차"] = train["일시"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])
submission["주차"] = submission["일시"].map(lambda x: datetime.datetime(x.year, x.month, x.day).isocalendar()[1])

In [21]:
# 계절 변수 생성
train['계절'] = ''

for i, month in enumerate(train['월']):
    if 3 <= month <= 5:
        train.at[i, '계절'] = '봄'
    elif 6 <= month <= 8:
        train.at[i, '계절'] = '여름'
    elif 9 <= month <= 11:
        train.at[i, '계절'] = '가을'
    else:
        train.at[i, '계절'] = '겨울'

# 결과 확인
print(train[['일시', '월', '계절']])


# 계절 변수 생성
submission['계절'] = ''

for i, month in enumerate(submission['월']):
    if 3 <= month <= 5:
        submission.at[i, '계절'] = '봄'
    elif 6 <= month <= 8:
        submission.at[i, '계절'] = '여름'
    elif 9 <= month <= 11:
        submission.at[i, '계절'] = '가을'
    else:
        submission.at[i, '계절'] = '겨울'

# 결과 확인
print(submission[['일시', '월', '계절']])

              일시   월  계절
0     1960-01-01   1  겨울
1     1960-01-02   1  겨울
2     1960-01-03   1  겨울
3     1960-01-04   1  겨울
4     1960-01-05   1  겨울
...          ...  ..  ..
23006 2022-12-27  12  겨울
23007 2022-12-28  12  겨울
23008 2022-12-29  12  겨울
23009 2022-12-30  12  겨울
23010 2022-12-31  12  겨울

[23011 rows x 3 columns]
            일시   월  계절
0   2023-01-01   1  겨울
1   2023-01-02   1  겨울
2   2023-01-03   1  겨울
3   2023-01-04   1  겨울
4   2023-01-05   1  겨울
..         ...  ..  ..
353 2023-12-20  12  겨울
354 2023-12-21  12  겨울
355 2023-12-22  12  겨울
356 2023-12-23  12  겨울
357 2023-12-24  12  겨울

[358 rows x 3 columns]


In [22]:
# '계절' 열에 대한 원핫인코딩 수행
one_hot_encoded = pd.get_dummies(train['계절'], prefix='계절')

# 기존 데이터프레임과 합치기
train = pd.concat([train, one_hot_encoded], axis=1)

# '계절' 열과 중복된 열 삭제
train.drop(['계절'], axis=1, inplace=True)

# 결과 확인
print(train)


# '계절' 열에 대한 원핫인코딩 수행
one_hot_encoded = pd.get_dummies(submission['계절'], prefix='계절')

# 기존 데이터프레임과 합치기
submission = pd.concat([submission, one_hot_encoded], axis=1)

# '계절' 열과 중복된 열 삭제
submission.drop(['계절'], axis=1, inplace=True)

# 결과 확인
print(train)
print(submission)

              일시  최고기온  최저기온   일교차  강수량  평균습도  평균풍속  일조합        일사합  \
0     1960-01-01   2.2  -5.2   7.4  0.4  68.3   1.7  6.7  13.038944   
1     1960-01-02   1.2  -5.6   6.8  0.4  87.7   1.3  0.0   4.181680   
2     1960-01-03   8.7  -2.1  10.8  0.0  81.3   3.0  0.0   4.181680   
3     1960-01-04  10.8   1.2   9.6  0.0  79.7   4.4  2.6   7.618827   
4     1960-01-05   1.3  -8.2   9.5  0.0  44.0   5.1  8.2  15.021913   
...          ...   ...   ...   ...  ...   ...   ...  ...        ...   
23006 2022-12-27   3.3  -7.3  10.6  0.1  69.8   1.8  8.8  10.250000   
23007 2022-12-28   0.1  -6.0   6.1  0.1  58.1   2.5  8.7  10.860000   
23008 2022-12-29   2.1  -7.8   9.9  0.0  56.3   1.7  9.0  10.880000   
23009 2022-12-30   2.3  -4.4   6.7  0.0  65.6   1.9  7.9  10.840000   
23010 2022-12-31   2.1  -5.1   7.2  0.0  65.5   1.4  1.1   4.160000   

             일조율  ...     년   월   일       Day sin   Day cos  주차  계절_가을  계절_겨울  \
0      55.602262  ...  1960   1   1  1.721336e-02  0.999852  53  F

In [23]:
train

,일시,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,...,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름
0,1960-01-01,2.2,-5.2,7.4,0.4,68.3,1.7,6.7,13.038944,55.602262,...,1960,1,1,1.721336e-02,0.999852,53,False,True,False,False
1,1960-01-02,1.2,-5.6,6.8,0.4,87.7,1.3,0.0,4.181680,2.124736,...,1960,1,2,3.442161e-02,0.999407,53,False,True,False,False
2,1960-01-03,8.7,-2.1,10.8,0.0,81.3,3.0,0.0,4.181680,2.124736,...,1960,1,3,5.161967e-02,0.998667,53,False,True,False,False
3,1960-01-04,10.8,1.2,9.6,0.0,79.7,4.4,2.6,7.618827,22.877209,...,1960,1,4,6.880243e-02,0.997630,1,False,True,False,False
4,1960-01-05,1.3,-8.2,9.5,0.0,44.0,5.1,8.2,15.021913,67.574842,...,1960,1,5,8.596480e-02,0.996298,1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23006,2022-12-27,3.3,-7.3,10.6,0.1,69.8,1.8,8.8,10.250000,91.700000,...,2022,12,27,-6.880243e-02,0.997630,52,False,True,False,False
23007,2022-12-28,0.1,-6.0,6.1,0.1,58.1,2.5,8.7,10.860000,90.600000,...,2022,12,28,-5.161967e-02,0.998667,52,False,True,False,False
23008,2022-12-29,2.1,-7.8,9.9,0.0,56.3,1.7,9.0,10.880000,93.800000,...,2022,12,29,-3.442161e-02,0.999407,52,False,True,False,False
23009,2022-12-30,2.3,-4.4,6.7,0.0,65.6,1.9,7.9,10.840000,82.300000,...,2022,12,30,-1.721336e-02,0.999852,52,False,True,False,False


In [24]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False


일교차 예측

In [27]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '계절_가을', '계절_겨울', '계절_봄', '계절_여름']
target_variable = '일교차'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의
xgb = XGBRegressor(random_state=2024,
                   n_estimators=1000,
                   learning_rate=0.01,
                   max_depth=10,
                   reg_lambda=3,
                   verbosity=1)  # verbosity를 1로 설정

# 모델 학습
xgb.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

In [29]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = xgb.predict(submission_data)

In [30]:
submission['일교차'] = submission_pred

In [31]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름,일교차
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False,10.524720
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False,7.932806
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False,8.805532
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False,7.970095
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False,8.815740
...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False,9.066409
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False,8.804821
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False,9.104725
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False,8.720455


일죠율 예측

In [32]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '일조율'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의
xgb = XGBRegressor(random_state=2024,
                   n_estimators=1000,
                   learning_rate=0.01,
                   max_depth=10,
                   reg_lambda=3,
                   verbosity=1)  # verbosity를 1로 설정

# 모델 학습
xgb.fit(train_x, train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

In [33]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = xgb.predict(submission_data)

In [34]:
submission['일조율'] = submission_pred

평균습도 예측

In [35]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '평균습도'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의
xgb = XGBRegressor(random_state=2024,
                   n_estimators=1000,
                   learning_rate=0.01,
                   max_depth=10,
                   reg_lambda=3,
                   verbosity=1)  # verbosity를 1로 설정

# 모델 학습
xgb.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

In [36]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = xgb.predict(submission_data)

In [37]:
submission['평균습도'] = submission_pred

강수량 예측

In [38]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '강수량'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의
xgb = XGBRegressor(random_state=2024,
                   n_estimators=1000,
                   learning_rate=0.01,
                   max_depth=10,
                   reg_lambda=3,
                   verbosity=1)  # verbosity를 1로 설정

# 모델 학습
xgb.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

In [39]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = xgb.predict(submission_data)

In [40]:
submission['강수량'] = submission_pred

In [41]:
submission

,일시,평균기온,년,월,일,Day sin,Day cos,주차,계절_가을,계절_겨울,계절_봄,계절_여름,일교차,일조율,평균습도,강수량
0,2023-01-01,0,2023,1,1,1.754990e-02,0.999846,52,False,True,False,False,10.524720,80.158203,61.789871,0.808444
1,2023-01-02,0,2023,1,2,3.509439e-02,0.999384,1,False,True,False,False,7.932806,76.290825,53.538097,0.695567
2,2023-01-03,0,2023,1,3,5.262807e-02,0.998614,1,False,True,False,False,8.805532,52.580582,58.516804,0.376787
3,2023-01-04,0,2023,1,4,7.014554e-02,0.997537,1,False,True,False,False,7.970095,85.777267,53.537613,0.310710
4,2023-01-05,0,2023,1,5,8.764140e-02,0.996152,1,False,True,False,False,8.815740,47.798111,57.378582,0.314997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
353,2023-12-20,0,2023,12,20,-7.014554e-02,0.997537,51,False,True,False,False,9.066409,68.916107,60.009907,0.719216
354,2023-12-21,0,2023,12,21,-5.262807e-02,0.998614,51,False,True,False,False,8.804821,61.143330,61.148014,0.685596
355,2023-12-22,0,2023,12,22,-3.509439e-02,0.999384,51,False,True,False,False,9.104725,56.236195,62.963573,0.726777
356,2023-12-23,0,2023,12,23,-1.754990e-02,0.999846,51,False,True,False,False,8.720455,71.204063,57.952099,0.611597


평균기온 예측

In [42]:
# 필요한 변수 선택 (예시로 선택된 변수들이므로 실제로 사용하는 변수에 맞게 수정 필요)
selected_features = ['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도', '강수량','계절_가을','계절_겨울','계절_봄','계절_여름']
target_variable = '평균기온'

# 선택된 변수들을 사용하여 훈련 데이터셋 준비
train_x = train[selected_features]
train_y = train[target_variable]

test_x = submission[selected_features]

# 모델 정의
xgb = XGBRegressor(random_state=2024,
                   n_estimators=1000,
                   learning_rate=0.01,
                   max_depth=10,
                   reg_lambda=3,
                   verbosity=1)  # verbosity를 1로 설정

# 모델 학습
xgb.fit(train_x, train_y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.01, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1000, n_jobs=None,
             num_parallel_tree=None, random_state=2024, ...)

In [43]:
# submission 데이터에서 필요한 변수만 남기기 (평균기온을 예측하기 위한 변수)
submission_data = submission[['년', '월', '일', 'Day sin', 'Day cos', '주차', '일교차', '일조율', '평균습도', '강수량','계절_가을','계절_겨울','계절_봄','계절_여름']]

# 예측 수행
submission_pred = xgb.predict(submission_data)

In [44]:
submission = pd.read_csv('./sample_submission.csv')
submission['평균기온']=submission_pred

In [45]:
submission

,일시,평균기온
0,2023-01-01,-0.919517
1,2023-01-02,-2.677367
2,2023-01-03,-1.996181
3,2023-01-04,-1.971666
4,2023-01-05,-2.691707
...,...,...
353,2023-12-20,-4.398148
354,2023-12-21,-3.158748
355,2023-12-22,-2.500262
356,2023-12-23,-3.810806


In [46]:
submission.to_csv('xgboost7.csv', index=False)